In [1]:
import nltk

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

from itertools import groupby
import pandas as pd

In [2]:
# Adapt those lines to your installation
jar_location = 'stanford-ner-2020-11-17/stanford-ner.jar'
model_location_3classes = 'stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz'
model_location_7classes = 'stanford-ner-2020-11-17/classifiers/english.muc.7class.distsim.crf.ser.gz'
st3 = StanfordNERTagger(model_location_3classes,jar_location,encoding='utf-8')
st7 = StanfordNERTagger(model_location_7classes,jar_location,encoding='utf-8')

In [3]:
text = '''Lovelace became close friends with her tutor Mary Somerville, who introduced her to Charles Babbage in 1833. She had a strong respect and affection for Somerville, and they corresponded for many years. Other acquaintances included the scientists Andrew Crosse, Sir David Brewster, Charles Wheatstone, Michael Faraday and the author Charles Dickens. She was presented at Court at the age of seventeen "and became a popular belle of the season" in part because of her "brilliant mind." By 1834 Ada was a regular at Court and started attending various events. She danced often and was able to charm many people, and was described by most people as being dainty, although John Hobhouse, Byron's friend, described her as "a large, coarse-skinned young woman but with something of my friend's features, particularly the mouth". This description followed their meeting on 24 February 1834 in which Ada made it clear to Hobhouse that she did not like him, probably because of the influence of her mother, which led her to dislike all of her father's friends. This first impression was not to last, and they later became friends.'''
tokenized_text = word_tokenize(text)
text_ner3 = st3.tag(tokenized_text)
text_ner7 = st7.tag(tokenized_text)

print("**** 7 classes ****")
for tag, chunk in groupby(text_ner7, lambda x:x[1]):
    if tag != "O":
        print("%-12s"%tag, " ".join(w for w, t in chunk))

**** 7 classes ****
PERSON       Mary Somerville
PERSON       Charles Babbage
DATE         1833
LOCATION     Somerville
PERSON       Andrew Crosse
PERSON       David Brewster
PERSON       Charles Wheatstone
PERSON       Michael Faraday
PERSON       Charles Dickens
DATE         1834
ORGANIZATION Ada
PERSON       John Hobhouse
PERSON       Byron
DATE         February 1834
ORGANIZATION Ada
PERSON       Hobhouse


In [4]:
import pandas as pd
import re

In [5]:
df = pd.read_csv('Cleaned_Ref_03.csv')
ss = ss = df['raw_ref'].values

In [6]:
s = 'This is a bad 2018'
match = re.search(r'[12]\d{3}', s)

if match:
    print(match.group(0))


2018


In [7]:
string = 'Vol 3, No.7'

def count_num(s):
    count = 0

    temp = list(s)
    for item in temp:
        if(item.isdigit()):
            count = count + 1
        else:
            pass
#    print (count)
    
    return count

In [8]:
#re.search(r'\d+', 'Vol 3, No.7').group()
re.sub('[^\d.,]' , '', 'Vol 3, No.7')

'3,.7'

In [13]:
ref_list = []
skipped_list = []

for k in range (len(ss)):
#    match = re.search(r'[12]\d{3}', ss[k])
        
    ref_dict = {'date':[], 'names':[], 'title':[], 'raw_ref':[]}
        
    name_set = False
    date_set = False
    pos = -1    
        
    match = re.search(r'[12]\d{3}', ss[k])

    if match != None:
        ref_dict['date'] = int(match.group(0))
        date_set = True
            
        #print (ss[k])

        pos = ss[k].find(match.group(0))
                
        tokenized_text = word_tokenize(ss[k][:pos])
        text_ner3 = st3.tag(tokenized_text)
     #   text_ner7 = st7.tag(tokenized_text)
        
        for tag, chunk in groupby(text_ner3, lambda x:x[1]):
            if tag != "O":
                if tag == 'PERSON':
                    if len(ref_dict['names']) == 0: 
                        ref_dict['names'] =  " ".join(w for w, t in chunk)
                        name_set = True
                    else:
                        ref_dict['names'] = ref_dict['names'] + ', ' +  " ".join(w for w, t in chunk)

                    #print(k, "%-12s"%tag, " ".join(w for w, t in chunk))
                        
    if date_set == True and name_set == True:
            ref_dict['raw_ref'] = ss[k]
            ref_dict['names'] =  ref_dict['names'][:-1]

            ref_list.append(ref_dict)
        
    else:
        print ('------- Ignored', k, pos, name_set, date_set, ss[k])
        skipped_list.append(ss[k])

------- Ignored 1 31 False True Conference on Neural Networks, 1990, pp.1385-1393.
------- Ignored 6 75 False True Proceedings of the IEEE International Joint Conference on Neural Networks, 1990, pp. 11-16.
------- Ignored 8 13 False True Vol 3, No.7, 2012
------- Ignored 10 12 False True Management, 1992.
------- Ignored 11 20 False True Science Publishers, 1993.
------- Ignored 13 93 False True Technical Analysis for Making Trading Decisions Using Historical Equity Market Data”, IAT04, 2004
------- Ignored 29 10 False True Bosq, D. (2000). Linear Process in Function Spaces: Theory and Applications. New York:
------- Ignored 43 9 False True Fan, Z. (2010) Statistical Issues and Developments in Time Series Analysis and Educational
------- Ignored 84 20 False True Phillips, P. C. B. (1987) Time series regression with a unit root. Econometrica, 55, 277-301.
------- Ignored 89 12 False True Quenouille (1949), Approximate tests of correlation in time series. Journal of Royal Statistical So

------- Ignored 566 68 False True [224]. Online available at http://www.it-weise.de/documents/files/BW2005QASD.pdf [accessed 2009-06-26]. [224] Steﬀen Bleul, Thomas Weise, and Kurt Geihs. An Ontology for Quality-Aware Service Discovery. Computer Systems Science Engineering, 21(4)(Special issue on “Engineering Design and Composition of Service-Oriented Applications”), July 2006. See also [223]. Online available at http://www.it-weise.de/documents/files/ BWG2006QASD.pdf [accessed 2009-06-26].
------- Ignored 590 211 False True [250] István Borgulya. A cluster-based evolutionary algorithm for the single machine total weighted tardiness-scheduling problem. Journal of Computing and Information Technology - CIT, 10(3):211–217, September 2002. Online available at http://cit. zesoi.fer.hr/downloadPaper.php?paper=384 [accessed 2008-04-07].
------- Ignored 613 150 False True [276] Jürgen Branke. Creating robust solutions by means of evolutionary algorithms. In Parallel Problem Solving from Nat

------- Ignored 1109 147 False True [804] Basilis Gidas. Nonstationary markov chains and convergence of the annealing algorithm. Journal of Statistical Physics, 39(1-2):73–131, April 1985. ISSN: 0022-4715 (Print) 1572-9613 (Online). doi:10.1007/BF01007975. Online available REFERENCES 663 at http://www.springerlink.com/content/x3821j328gv33085/fulltext.pdf [accessed 2008-03-26].
------- Ignored 1184 214 False True [884] Lin Han and Xingshi He. A novel opposition-based particle swarm optimization for noisy problems. In ICNC’07: Proceedings of the Third International Conference on Natural Computation, volume 3, pages 624–629, 2007. doi:10.1109/ICNC.2007.119. In proceedings [997].
------- Ignored 1205 120 False True [908]. Online available at http://www.mcs.utulsa.edu/~rogerw/papers/Haynes-icmas95.pdf and http:// citeseer.ist.psu.edu/1958.html [accessed 2007-10-03]. [908] Thomas D. Haynes, Roger L. Wainwright, and Sandip Sen. Evolving cooperating strategies. In Victor Lesser, editor, Proce

------- Ignored 1301 1 False True [1910] Shana Shiang-Fong Smith. Using multiple genetic operators to reduce premature convergence in genetic assembly planning. Computers in Industry, 54(1):35–49, May 2004. ISSN: 0166-3615. doi:10.1016/j.compind.2003.08.001.
------- Ignored 1302 1 False True [1911] Stephen Smith and Stefano Cagnoni, editors. Medical Applications of Genetic and Evolutionary Computation (MedGEC) Workshop 2008, July 12, 2008, Renaissance Atlanta Hotel Downtown, 590 West Peachtree Street NW, Atlanta, Georgia 30308 USA. ACM Press, New York, NY, USA. ISBN: 978-1-60558-131-6. Part of GECCO 2008, see also [1117] and http://www.elec.york.ac.uk/intsys/events/ MedGEC2008/home.htm [accessed 2008-07-21].
------- Ignored 1303 1 False True [1912] Stephen Frederick Smith. A Learning System based on Genetic Adaptive Algorithms. PhD thesis, University of Pittsburgh, Pittsburgh, PA, USA, 1980. Order No. AAI8112638, University Microﬁlms No. 81-12638.
------- Ignored 1304 1 False True [191

------- Ignored 1323 1 False True [1932] Lee Spector. Automatic Quantum Computer Programming – A Genetic Programming Approach. Genetic Programming. Kluwer Academic Publishers / Springer Science+Business Media, New York, paperback edition 2007 edition, June 2004. ISBN: 0-3873-6496-X, 0-3873-6791-8, 978-0-38736-496-4, 978-0-38736-791-0. Library of Congress Control Number: 2006931640. Series editor: John Koza. Partly online available at http://books.google.de/books?id=mEKfvtxmn9MC [accessed 2008-05-04]. REFERENCES 759
------- Ignored 1324 1 False True [1933] Lee Spector and Alan Robinson. Multi-type, self-adaptive genetic programming as an agent creation tool. In GECCO 2002: Proceedings of the Bird of a Feather Workshops, Genetic and Evolutionary Computation Conference, pages 73–80, 2002. In proceedings [154]. Online available at http://citeseer.ist.psu.edu/614297.html and http://hampshire.edu/lspector/pubs/ecomas2002-spector-toappear.pdf [accessed 2007-12-25].
------- Ignored 1325 1 Fals

...

------- Ignored 53335 36 False True Multi-class deep boosting. In NIPS, 2014.
------- Ignored 53336 31 False True Neural Networks, 8(3):630–645, 1997.
------- Ignored 53339 7 False True 79–88, 2003.
------- Ignored 53340 22 False True In NIPS, pp. 855–863, 2014.
------- Ignored 53343 6 False True COLT, 2015.
------- Ignored 53344 9 False True 487–494, 2001.
------- Ignored 53345 29 False True A theoretical view. In AAAI, 2016.
------- Ignored 53347 41 False True Going deeper with convolutions. In CVPR, 2015.
------- Ignored 53348 6 False True COLT, 2016.
------- Ignored 53352 178 False True “TensorFlow: a system for large-scale machine learning,” in Proceedings of the 12th USENIX conference on operating systems design and implementation, Savannah, GA, November 2–4, 2016 (Berkeley, CA: OSDI’16) 265–283.
------- Ignored 53354 73 False True Preprint repository name [Preprint]. Available at: https://arxiv.org/abs/1901.
------- Ignored 53356 10 False True Bayer J. (2015). Learning sequence 

------- Ignored 53634 121 False True 47. Kriminger E, Principe JC and Lakshminarayan C. Nearest neighbor distributions for imbalanced classification. In: The 2012 international joint conference on neural networks (IJCNN), Brisbane, QLD, Australia, 10–15 June 2012, pp.1–5. New York: IEEE.
------- Ignored 53636 97 False True 49. Tomašev N and Mladenić D. Class imbalance and the curse of minority hubs. Knowl Based Syst 2013; 53: 157–172.
------- Ignored 53644 160 False True 57. Ramentol E, Vluymans S, Verbiest N, et al. IFROWANN: imbalanced fuzzy-rough ordered weighted average nearest neighbor classification. IEEE Trans Fuzzy Syst 2015; 23: 1622–1637.
------- Ignored 53655 204 False True 68. López V, Fernández A, Moreno-Torres JG, et al. Analysis of preprocessing vs. cost-sensitive learning for imbalanced classification: open problems on intrinsic data characteristics. Expert Syst Appl 2012; 39: 6585–6608.
------- Ignored 53714 1 False True (2018). Double/debiased machine learning for

------- Ignored 54182 2 False True ——2000 The proﬁtability of intra-day FX trading using technical indicators Working Paper 35/00 Judge Institute of
------- Ignored 54183 0 False True 2000 Computational learning techniques for intraday FX trading using popular technical indicators Working Paper 30/00
------- Ignored 54184 56 False True Judge Institute of Management, University of Cambridge (2001
------- Ignored 54186 13 False True Gencay et al 1998 Real time trading models and the statistical properties of foreign exchange rates Working Paper Olsen & Associates www.olsen.ch
------- Ignored 54191 2 False True ——1980 Adaptive algorithms for discovering and using general patterns in growing knowledge-bases Int. J. Policy Analysis
------- Ignored 54192 2 False True ——1986 Escaping brittleness: the possibilities of general-purpose learning algorithms applied to parallel rule-based systems
------- Ignored 54193 2 False True ——1992 Adaptation in Natural and Artiﬁcial Systems (Cambridge, MA: M

------- Ignored 54501 118 False True IEEE/PES transmission and distribution conference and exhibition: Asia and Pacific, pp 1–6, Dalian, China, Aug 15–18, 2005
------- Ignored 54505 29 False True Proceedings of the AAAI Fall 1997 symposium on communicative action in humans and machines, pp 67–72, Cambridge, MA, USA, Nov 8–10, 1997
------- Ignored 54507 22 False True Diego, CA, Mar 10–13, 1998
------- Ignored 54511 0 False True 2003 IEEE swarm intelligence symposium (SIS’03), pp 80–87, Indianapolis, IN, USA, April 24–26, 2003
------- Ignored 54513 121 False True Proceedings of the IEEE international conference on evolutionary computation, pp 340–347, Washington, DC, USA, July 6–9, 2004
------- Ignored 54518 22 False True York, USA, Oct 12–14, 2003
------- Ignored 54520 34 False True 621–630, Granada, Spain, 7–11 Dec 2002
------- Ignored 54523 34 False True 55, Chongqing, China, Sept 21–24, 2005
------- Ignored 54526 6 False True (GECCO2004), pp 117–128, Seattle, WA, USA, June 26–30, 200

------- Ignored 54785 21 False True Pattern Recognition, 2018.
------- Ignored 54795 6 False True [10] (2012) MATLAB GlobalOptimization Toolbox [Online]. Available: http//www.mathworks.com/products/globaloptimization/
------- Ignored 54796 6 False True [11] (2012) MATLAB Mixed Integer Optimization Problems, [Online]. Available: http://www.mathworks.com/help/toolbox/gads/bs1cibj.html#b s1cihn.
------- Ignored 54798 31 False True Admati, A., and P. Pfleiderer (1988): “A Theory of Intraday Patterns: Volume and
------- Ignored 54804 13 False True BATS Europe (2009): “Participant Notice, Amendments to the Rule Book and Participant Manual,” BATS Trading, http://www.batstrading.co.uk/resources/notices/ participant/BATSEuro_PN09_02.pdf.
------- Ignored 54808 29 False True //www.sec.gov/news/testimony/2009/ts102809jab.htm.
------- Ignored 54813 6 False True CESR (2009): “Waivers from Pre-trade Transparency Obligations under the Markets in
------- Ignored 54814 17 False True Chakravorti, B. (199

In [14]:
len(ref_list), len(skipped_list)

(48062, 6958)

In [16]:
df = pd.DataFrame.from_dict(ref_list)
df.to_csv('Parsed_Raw_Ref_48062.csv')

In [17]:
df = pd.DataFrame(skipped_list)
df.to_csv('skipped_raw_ref_6968.csv')